1. How many orders are cancelled and what’s the total value?

2. What’s the percentage of orders cancelled before payment and after payment?

3. Is cancellation rate higher for certain payment channels?

4. How many percent of orders are re-ordered from the same shop, and how long after cancellation do they re-order (mean)? 

Information about the dataset

All calculations should exclude orders from Shop ID 111111111.

Order Status:

- Paid = 2, 11, 12, 13
- Unpaid and Cancelled = 6
- Paid then Cancelled = 7, 8, 9, 10

In [328]:
import pandas as pd 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [329]:
file = "Training Workshop Data File - Sample Orders.xlsx"

df_order = pd.read_excel(open(file, 'rb'), sheet_name="Order Data")  
df_pay   = pd.read_excel(open(file, 'rb'), sheet_name="Payments Data")  



df_order.head()
df_order.shape
df_order.info()
df_pay.head()
df_pay.shape

,orderid,shopid,userid,checkoutid,total_price,currency,status,created_time,payment_time,cancel_time
0,6505222765,900457733,100223545,1044269436,147.168235,SGD,2,14/11/2019 21:33,2019-11-16 12:54:00,NaT
1,6135926959,586311226,100223545,1211270092,33.169735,SGD,2,13/12/2019 19:02,2019-12-13 21:33:00,NaT
2,5513066971,949679556,100223545,1676514901,24.112195,SGD,2,31/12/2019 17:51,2019-12-31 17:55:00,NaT
3,5123701563,894942828,100223545,1211270092,75.913637,SGD,6,13/12/2019 19:02,NaT,2019-12-15 21:38:00
4,7311653202,389950494,100223545,1880489412,14.455363,SGD,6,22/12/2019 14:47,NaT,2019-12-25 11:44:00


(7987, 10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7987 entries, 0 to 7986
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   orderid       7987 non-null   int64         
 1   shopid        7987 non-null   int64         
 2   userid        7987 non-null   int64         
 3   checkoutid    7987 non-null   int64         
 4   total_price   7987 non-null   float64       
 5   currency      7987 non-null   object        
 6   status        7987 non-null   int64         
 7   created_time  7987 non-null   object        
 8   payment_time  6284 non-null   datetime64[ns]
 9   cancel_time   2628 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(5), object(2)
memory usage: 624.1+ KB


,checkoutid,total_price,currency,payment_method,payment_time
0,123016653,1.176036,SGD,Credit Card,2019-12-12 19:03:00
1,128375081,3.461700,SGD,Credit Card,2019-12-28 00:13:00
2,150417433,3.188444,SGD,Credit Card,2019-12-10 11:30:00
3,169495571,1.664013,SGD,Credit Card,2019-11-05 19:11:00
4,205419935,1.244183,SGD,Credit Card,2019-11-12 02:05:00


(7838, 5)

In [330]:
#exclude orders from Shop ID 111111111. 
df_order = df_order[df_order["shopid"]!=111111111]

df_order.shape


(7911, 10)

1. How many orders are cancelled and what’s the total value?

In [331]:
#cancelled orders

df_cancelled = df_order[df_order["status"].isin([6,7,8,9,10])]
print("Cancelled Orders : {:,}".format(df_cancelled["orderid"].nunique()))
print("Sum of Cancelled Orders in Amount : ${:,.0f}".format(df_cancelled["total_price"].sum()))

Cancelled Orders : 2,555
Sum of Cancelled Orders in Amount : $194,769


2. What’s the percentage of orders cancelled before payment and after payment?

In [332]:
df_cancel_status = df_cancelled.groupby("status")["orderid"].count().reset_index()
df_cancel_status



,status,orderid
0,6,1703
1,7,205
2,8,192
3,9,222
4,10,233


In [333]:
df_cancel_before = df_cancel_status[df_cancel_status["status"]==6]["orderid"].sum()
df_cancel_after  = df_cancel_status[df_cancel_status["status"]!=6]["orderid"].sum()

all_orders= df_cancel_status["orderid"].sum()

# print(df_cancel_before)
# print(df_cancel_after)

print("#Orders Cancelled Before {:,}, ({:.1%})".format(df_cancel_before, df_cancel_before/all_orders))
print("#Orders Cancelled After  {:,}, ({:.1%})".format(df_cancel_after, df_cancel_after/all_orders))



#Orders Cancelled Before 1,703, (66.7%)
#Orders Cancelled After  852, (33.3%)


3. Is cancellation rate higher for certain payment channels?

In [334]:
df_merge = df_order.merge(df_pay, on="checkoutid", how="left")
print("Rows, Columns: {},{}".format(df_merge.shape[0], df_merge.shape[1]))
df_merge.head()

df_merge.info()

Rows, Columns: 7911,14


,orderid,shopid,userid,checkoutid,total_price_x,currency_x,status,created_time,payment_time_x,cancel_time,total_price_y,currency_y,payment_method,payment_time_y
0,6505222765,900457733,100223545,1044269436,147.168235,SGD,2,14/11/2019 21:33,2019-11-16 12:54:00,NaT,NaN,NaN,NaN,NaT
1,6135926959,586311226,100223545,1211270092,33.169735,SGD,2,13/12/2019 19:02,2019-12-13 21:33:00,NaT,NaN,NaN,NaN,NaT
2,5513066971,949679556,100223545,1676514901,24.112195,SGD,2,31/12/2019 17:51,2019-12-31 17:55:00,NaT,24.112195,SGD,Credit Card,2019-12-31 17:55:00
3,5123701563,894942828,100223545,1211270092,75.913637,SGD,6,13/12/2019 19:02,NaT,2019-12-15 21:38:00,NaN,NaN,NaN,NaT
4,7311653202,389950494,100223545,1880489412,14.455363,SGD,6,22/12/2019 14:47,NaT,2019-12-25 11:44:00,14.455363,SGD,Credit Card,NaT


<class 'pandas.core.frame.DataFrame'>
Int64Index: 7911 entries, 0 to 7910
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   orderid         7911 non-null   int64         
 1   shopid          7911 non-null   int64         
 2   userid          7911 non-null   int64         
 3   checkoutid      7911 non-null   int64         
 4   total_price_x   7911 non-null   float64       
 5   currency_x      7911 non-null   object        
 6   status          7911 non-null   int64         
 7   created_time    7911 non-null   object        
 8   payment_time_x  6208 non-null   datetime64[ns]
 9   cancel_time     2555 non-null   datetime64[ns]
 10  total_price_y   7803 non-null   float64       
 11  currency_y      7803 non-null   object        
 12  payment_method  7803 non-null   object        
 13  payment_time_y  6134 non-null   datetime64[ns]
dtypes: datetime64[ns](3), float64(2), int64(5), object(4)
me

In [335]:
#find repeated columns
col_dup = df_merge.columns[df_merge.columns.str.contains("_y")]
col_dup

Index(['total_price_y', 'currency_y', 'payment_time_y'], dtype='object')

In [336]:
#drop the repeated columns
df_merge.drop(col_dup, axis="columns", inplace=True)
df_merge.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 7911 entries, 0 to 7910
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   orderid         7911 non-null   int64         
 1   shopid          7911 non-null   int64         
 2   userid          7911 non-null   int64         
 3   checkoutid      7911 non-null   int64         
 4   total_price_x   7911 non-null   float64       
 5   currency_x      7911 non-null   object        
 6   status          7911 non-null   int64         
 7   created_time    7911 non-null   object        
 8   payment_time_x  6208 non-null   datetime64[ns]
 9   cancel_time     2555 non-null   datetime64[ns]
 10  payment_method  7803 non-null   object        
dtypes: datetime64[ns](2), float64(1), int64(5), object(3)
memory usage: 741.7+ KB


In [337]:
#rename the variables
df_merge.rename({"total_price_x": "total_price", "currency_x":"currency" ,"payment_time_x": "payment_time"},axis="columns", inplace=True)
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7911 entries, 0 to 7910
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   orderid         7911 non-null   int64         
 1   shopid          7911 non-null   int64         
 2   userid          7911 non-null   int64         
 3   checkoutid      7911 non-null   int64         
 4   total_price     7911 non-null   float64       
 5   currency        7911 non-null   object        
 6   status          7911 non-null   int64         
 7   created_time    7911 non-null   object        
 8   payment_time    6208 non-null   datetime64[ns]
 9   cancel_time     2555 non-null   datetime64[ns]
 10  payment_method  7803 non-null   object        
dtypes: datetime64[ns](2), float64(1), int64(5), object(3)
memory usage: 741.7+ KB


In [338]:
# - Paid = 2, 11, 12, 13
# - Unpaid and Cancelled = 6
# - Paid then Cancelled = 7, 8, 9, 10

status_paid =[2,11,12,13]

df_merge.groupby(["status","payment_method"])["orderid"].count().reset_index()




,status,payment_method,orderid
0,2,Bank Transfer,199
1,2,Credit Card,1072
2,2,ShopeePay,53
3,6,Bank Transfer,244
4,6,Credit Card,1338
5,6,ShopeePay,87
6,7,Bank Transfer,26
7,7,Credit Card,167
8,7,ShopeePay,8
9,8,Bank Transfer,32


In [339]:
def check_cancelled(df):
    status = df["status"]
    if status in status_paid :
        return "not cancelled"
    else:
        return "cancelled"


df_merge["is_cancelled"] = df_merge.apply(check_cancelled, axis="columns") 
df_merge.head()

,orderid,shopid,userid,checkoutid,total_price,currency,status,created_time,payment_time,cancel_time,payment_method,is_cancelled
0,6505222765,900457733,100223545,1044269436,147.168235,SGD,2,14/11/2019 21:33,2019-11-16 12:54:00,NaT,NaN,not cancelled
1,6135926959,586311226,100223545,1211270092,33.169735,SGD,2,13/12/2019 19:02,2019-12-13 21:33:00,NaT,NaN,not cancelled
2,5513066971,949679556,100223545,1676514901,24.112195,SGD,2,31/12/2019 17:51,2019-12-31 17:55:00,NaT,Credit Card,not cancelled
3,5123701563,894942828,100223545,1211270092,75.913637,SGD,6,13/12/2019 19:02,NaT,2019-12-15 21:38:00,NaN,cancelled
4,7311653202,389950494,100223545,1880489412,14.455363,SGD,6,22/12/2019 14:47,NaT,2019-12-25 11:44:00,Credit Card,cancelled


In [340]:
df_payment_cancel = df_merge[df_merge["is_cancelled"]=="cancelled"].groupby(["is_cancelled","payment_method"])["orderid"].agg(["count"]).reset_index()
df_payment_cancel

,is_cancelled,payment_method,count
0,cancelled,Bank Transfer,372
1,cancelled,Credit Card,2014
2,cancelled,ShopeePay,121


In [341]:
n_cancelled = sum(df_payment_cancel["count"])
print(n_cancelled)


df_payment_cancel[df_payment_cancel["is_cancelled"]=="cancelled"]
df_payment_cancel["count"] = df_payment_cancel["count"].astype("float64")
df_cancel_pct = df_payment_cancel.groupby(level=0).sum()/n_cancelled

df_cancel_pct = round(df_cancel_pct*100,1)
df_cancel_pct.rename({"count":"%"} ,axis=1, inplace=True)
df_cancel_pct


2507


,is_cancelled,payment_method,count
0,cancelled,Bank Transfer,372
1,cancelled,Credit Card,2014
2,cancelled,ShopeePay,121


,%
0,14.8
1,80.3
2,4.8


Most of the cancelled transactions are from credit cards.

If we consider the percentage of cancelled transactions, by payment types:

In [342]:
pay_cancel = df_merge.groupby(['payment_method', 'is_cancelled']).agg({'orderid': 'count'})
pay_cancel

#calc sum and div
pay_cancel_pct = pay_cancel.groupby(level=0).apply(lambda x:
                                                100 * x / float(x.sum()))

pay_cancel_pct = round(pay_cancel_pct,1)
pay_cancel_pct.rename({"orderid":"%"} ,axis=1, inplace=True)
pay_cancel_pct

orderid
payment_method is_cancelled          
Bank Transfer  cancelled          372
               not cancelled      773
Credit Card    cancelled         2014
               not cancelled     4265
ShopeePay      cancelled          121
               not cancelled      258

%
payment_method is_cancelled       
Bank Transfer  cancelled      32.5
               not cancelled  67.5
Credit Card    cancelled      32.1
               not cancelled  67.9
ShopeePay      cancelled      31.9
               not cancelled  68.1

Bank Transfers have the highest cancellation rate among the 3 payment methods.


In [343]:
#from workshop example 

cancel = df_merge.groupby(['payment_method','is_cancelled'])['orderid'].nunique()
cancel 
cancel/cancel.sum(level='payment_method')

payment_method  is_cancelled 
Bank Transfer   cancelled         372
                not cancelled     773
Credit Card     cancelled        2014
                not cancelled    4265
ShopeePay       cancelled         121
                not cancelled     258
Name: orderid, dtype: int64

payment_method  is_cancelled 
Bank Transfer   cancelled        0.324891
                not cancelled    0.675109
Credit Card     cancelled        0.320752
                not cancelled    0.679248
ShopeePay       cancelled        0.319261
                not cancelled    0.680739
Name: orderid, dtype: float64

WIP

4. How many percent of orders are re-ordered from the same shop, and how long after cancellation do they re-order (mean)? 

In [344]:
df_merge.head()

df_merge[["orderid","shopid","userid","checkoutid", "created_time","payment_time","is_cancelled"]].sort_values(["shopid","userid"])

,orderid,shopid,userid,checkoutid,total_price,currency,status,created_time,payment_time,cancel_time,payment_method,is_cancelled
0,6505222765,900457733,100223545,1044269436,147.168235,SGD,2,14/11/2019 21:33,2019-11-16 12:54:00,NaT,NaN,not cancelled
1,6135926959,586311226,100223545,1211270092,33.169735,SGD,2,13/12/2019 19:02,2019-12-13 21:33:00,NaT,NaN,not cancelled
2,5513066971,949679556,100223545,1676514901,24.112195,SGD,2,31/12/2019 17:51,2019-12-31 17:55:00,NaT,Credit Card,not cancelled
3,5123701563,894942828,100223545,1211270092,75.913637,SGD,6,13/12/2019 19:02,NaT,2019-12-15 21:38:00,NaN,cancelled
4,7311653202,389950494,100223545,1880489412,14.455363,SGD,6,22/12/2019 14:47,NaT,2019-12-25 11:44:00,Credit Card,cancelled


,orderid,shopid,userid,checkoutid,created_time,payment_time,is_cancelled
7174,5887384027,100264691,916554285,1806364179,12/12/2019 07:50,2019-12-13 01:32:00,not cancelled
4499,4506038116,100464073,611554042,1059688064,05/12/2019 13:37,NaT,cancelled
1596,1526942168,100531365,283568769,1215585382,01/10/2019 11:44,2019-10-03 11:20:00,not cancelled
4959,3563024034,100728385,667880870,1493749164,10/10/2019 16:25,2019-10-10 16:28:00,not cancelled
650,9510247544,100823882,176830897,1637848990,08/10/2019 16:28,NaT,cancelled
...,...,...,...,...,...,...,...
3726,7923134997,999467686,523758062,1694279965,05/10/2019 00:16,2019-10-06 07:45:00,not cancelled
6172,7613607926,999612455,801490536,1389090019,24/11/2019 22:37,2019-11-25 03:20:00,not cancelled
1230,1382065621,999736550,241476014,1795202716,29/11/2019 21:09,2019-12-01 19:57:00,not cancelled
4074,4521197700,999820287,560189570,1129779569,16/10/2019 20:03,2019-10-18 01:15:00,cancelled


How many users have >1 order per shop?

In [345]:
df_user_orders = df_merge.groupby(["shopid","userid"])[["orderid"]].count().reset_index()
df_user_orders.rename({"orderid":"count"}, axis="columns",inplace=True)

#df_user_orders.info()

df_user_orders


,shopid,userid,count
0,100264691,916554285,1
1,100464073,611554042,1
2,100531365,283568769,1
3,100728385,667880870,1
4,100823882,176830897,1
...,...,...,...
7832,999467686,523758062,1
7833,999612455,801490536,1
7834,999736550,241476014,1
7835,999820287,560189570,1


In [346]:
#df_user_orders[df_user_orders["count"]>1].shape

print("{} users have >1 order from a each shop".format(df_user_orders[df_user_orders["count"]>1].shape[0]))

51 users have >1 order from a each shop


Check how many cancelled orders these 51 users have and compare 

In [347]:
df_orders_cancel = df_merge[df_merge["is_cancelled"]=="cancelled"][["shopid","userid","orderid", "created_time","payment_time"]]
df_orders_not_cancel = df_merge[df_merge["is_cancelled"]!="cancelled"][["shopid","userid","orderid", "created_time","payment_time"]]

df_orders_cancel.shape

df_orders_cancel.info()
df_orders_cancel.head()


df_orders_not_cancel.shape

df_orders_not_cancel.head()

(2555, 5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2555 entries, 3 to 7907
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   shopid        2555 non-null   int64         
 1   userid        2555 non-null   int64         
 2   orderid       2555 non-null   int64         
 3   created_time  2555 non-null   object        
 4   payment_time  852 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 119.8+ KB


,shopid,userid,orderid,created_time,payment_time
3,894942828,100223545,5123701563,13/12/2019 19:02,NaT
4,389950494,100223545,7311653202,22/12/2019 14:47,NaT
5,817856614,100223545,9459653001,14/11/2019 21:33,2019-11-16 12:54:00
18,778083005,101504702,3226040397,01/12/2019 06:12,NaT
19,127684957,101730236,7293234701,30/12/2019 14:37,NaT


(5356, 5)

,shopid,userid,orderid,created_time,payment_time
0,900457733,100223545,6505222765,14/11/2019 21:33,2019-11-16 12:54:00
1,586311226,100223545,6135926959,13/12/2019 19:02,2019-12-13 21:33:00
2,949679556,100223545,5513066971,31/12/2019 17:51,2019-12-31 17:55:00
6,373007278,100223545,5791188797,28/10/2019 08:27,2019-10-30 01:21:00
7,299238747,100223545,7926278218,14/11/2019 12:55,2019-11-14 12:57:00


In [348]:
df_users_2 = df_user_orders.merge(df_orders_not_cancel, on=["shopid","userid"], how="left")

df_users_2 = df_users_2.merge(df_orders_cancel, on=["shopid","userid"], how="left")
df_users_2.rename({"created_time_y":"cancelled_order_created_time"}, axis=1, inplace=True)
df_users_2.head(10)
df_users_2.info()



,shopid,userid,count,orderid_x,created_time_x,payment_time_x,orderid_y,cancelled_order_created_time,payment_time_y
0,100264691,916554285,1,5.887384e+09,12/12/2019 07:50,2019-12-13 01:32:00,NaN,NaN,NaT
1,100464073,611554042,1,NaN,NaN,NaT,4.506038e+09,05/12/2019 13:37,NaT
2,100531365,283568769,1,1.526942e+09,01/10/2019 11:44,2019-10-03 11:20:00,NaN,NaN,NaT
3,100728385,667880870,1,3.563024e+09,10/10/2019 16:25,2019-10-10 16:28:00,NaN,NaN,NaT
4,100823882,176830897,1,NaN,NaN,NaT,9.510248e+09,08/10/2019 16:28,NaT
5,100911245,969055358,1,7.872772e+09,26/11/2019 08:08,2019-11-29 05:17:00,NaN,NaN,NaT
6,101290422,942227355,1,1.395147e+09,10/12/2019 16:21,2019-12-11 16:02:00,NaN,NaN,NaT
7,101347272,274289630,1,9.646539e+09,28/11/2019 23:28,2019-11-29 02:19:00,NaN,NaN,NaT
8,101580151,747097213,1,6.953234e+09,04/10/2019 05:51,2019-10-04 23:39:00,NaN,NaN,NaT
9,101650594,628261101,1,NaN,NaN,NaT,1.225461e+09,29/10/2019 11:29,2019-10-31 10:33:00


<class 'pandas.core.frame.DataFrame'>
Int64Index: 7885 entries, 0 to 7884
Data columns (total 9 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   shopid                        7885 non-null   int64         
 1   userid                        7885 non-null   int64         
 2   count                         7885 non-null   int64         
 3   orderid_x                     5363 non-null   float64       
 4   created_time_x                5363 non-null   object        
 5   payment_time_x                5363 non-null   datetime64[ns]
 6   orderid_y                     2566 non-null   float64       
 7   cancelled_order_created_time  2566 non-null   object        
 8   payment_time_y                857 non-null    datetime64[ns]
dtypes: datetime64[ns](2), float64(2), int64(3), object(2)
memory usage: 616.0+ KB


In [349]:
#count how many uncancelled orders are after a cancelled order
print("orders re-ordered after cancellation: ")
df_users_2[df_users_2["created_time_x"]>df_users_2["cancelled_order_created_time"]]["orderid_x"].nunique()

orders re-ordered after cancellation: 


20

In [350]:
df_re_ordered = df_users_2[df_users_2["created_time_x"]>df_users_2["cancelled_order_created_time"]]
df_re_ordered.head()

df_re_ordered.shape

,shopid,userid,count,orderid_x,created_time_x,payment_time_x,orderid_y,cancelled_order_created_time,payment_time_y
461,154396809,481323800,5,1.636792e+09,22/11/2019 22:21,2019-11-22 22:22:00,6.795149e+09,21/11/2019 22:10,NaT
462,154396809,481323800,5,1.636792e+09,22/11/2019 22:21,2019-11-22 22:22:00,5.693226e+09,19/12/2019 02:56,NaT
463,154396809,481323800,5,1.636792e+09,22/11/2019 22:21,2019-11-22 22:22:00,3.822483e+09,21/11/2019 18:23,2019-11-21 21:34:00
1925,315658623,652903759,2,5.382642e+09,24/10/2019 09:18,2019-10-27 04:54:00,2.943183e+09,12/12/2019 13:19,NaT
2499,383755396,106380451,3,3.322028e+09,11/11/2019 18:13,2019-11-11 18:20:00,9.261368e+09,11/11/2019 09:41,2019-11-11 18:02:00


(22, 9)

In [351]:
#convert to dates
df_re_ordered["created_time_x"] = pd.to_datetime(df_re_ordered["created_time_x"])
df_re_ordered["cancelled_order_created_time"] = pd.to_datetime(df_re_ordered["cancelled_order_created_time"])

df_re_ordered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22 entries, 461 to 7818
Data columns (total 9 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   shopid                        22 non-null     int64         
 1   userid                        22 non-null     int64         
 2   count                         22 non-null     int64         
 3   orderid_x                     22 non-null     float64       
 4   created_time_x                22 non-null     datetime64[ns]
 5   payment_time_x                22 non-null     datetime64[ns]
 6   orderid_y                     22 non-null     float64       
 7   cancelled_order_created_time  22 non-null     datetime64[ns]
 8   payment_time_y                13 non-null     datetime64[ns]
dtypes: datetime64[ns](4), float64(2), int64(3)
memory usage: 1.7 KB


<ipython-input-351-6a2e8120175d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_re_ordered["created_time_x"] = pd.to_datetime(df_re_ordered["created_time_x"])
<ipython-input-351-6a2e8120175d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_re_ordered["cancelled_order_created_time"] = pd.to_datetime(df_re_ordered["cancelled_order_created_time"])


In [352]:
(df_re_ordered["created_time_x"]- df_re_ordered["cancelled_order_created_time"]).mean()

Timedelta('6 days 03:15:46.363636')

In [353]:
#try a different method 

#df_users_2 = df_user_orders.merge(df_orders_not_cancel, on=["shopid","userid"], how="left")

df_2 = df_orders_not_cancel.merge(df_orders_cancel, on=["shopid","userid"], how="left")
df_2.rename({"created_time_y":"cancelled_order_created_time"}, axis=1, inplace=True)
#df_2.head(10)
#df_2.info()


In [354]:
#count how many uncancelled orders are after a cancelled order
print("orders re-ordered after cancellation: ")
df_2[df_2["created_time_x"]>df_2["cancelled_order_created_time"]]["orderid_x"].count()

df_2["created_time_x"] = pd.to_datetime(df_2["created_time_x"])
df_2["cancelled_order_created_time"] = pd.to_datetime(df_2["cancelled_order_created_time"])

#df_re_ordered.info()



df_re_ordered2 = df_2[df_2["created_time_x"]>df_2["cancelled_order_created_time"]]
#df_re_ordered2.head()

df_re_ordered2.shape

(df_re_ordered2["created_time_x"]- df_re_ordered2["cancelled_order_created_time"]).mean()

orders re-ordered after cancellation: 


22

(20, 8)

Timedelta('28 days 07:31:03')

In [365]:
df_re_ordered2.head()

,shopid,userid,orderid_x,created_time_x,payment_time_x,orderid_y,cancelled_order_created_time,payment_time_y
36,383755396,106380451,3322028379,2019-11-11 18:13:00,2019-11-11 18:20:00,9.261368e+09,2019-11-11 09:41:00,2019-11-11 18:02:00
47,626078476,108425329,9549687038,2019-11-18 10:59:00,2019-11-18 11:30:00,2.209562e+09,2019-11-18 00:59:00,2019-11-18 01:00:00
73,967480130,113340269,4823971472,2019-10-26 15:09:00,2019-10-26 15:25:00,1.794071e+09,2019-10-23 06:19:00,2019-10-24 04:01:00
78,872033164,114564157,1938981468,2019-12-23 21:30:00,2019-12-24 09:38:00,2.289228e+09,2019-12-21 04:33:00,2019-12-21 20:51:00
79,788847405,114564157,6101181691,2019-12-23 21:30:00,2019-12-23 21:35:00,7.376344e+09,2019-10-30 02:51:00,2019-10-31 02:05:00


In [364]:
#from workshop solution 

cancel = df_merge.groupby(['payment_method','is_cancelled'])['orderid'].nunique()
cancel

#find all cancelled orders
cancelledorders = df_order[df_order['status'].isin([6,7,8,9,10])]
print("cancelled orders: "+str(cancelledorders.shape[0]))

#find not cancelled orders
notcancelledorders = df_order[~df_order['status'].isin([6,7,8,9,10])]
print("not cancelled orders: " + str(notcancelledorders.shape[0]))


compare = cancelledorders.merge(notcancelledorders,how='inner', on=['userid','shopid'])
#compare

compare['created_time_y']=pd.to_datetime(compare['created_time_y'])
#compare.dtypes

reorder=compare[compare['created_time_y']>=compare['cancel_time_x']]
print("re-ordered :" +str(reorder.shape[0]))

reorder_time = (reorder['created_time_y']-reorder['cancel_time_x']).mean()
reorder_time

payment_method  is_cancelled 
Bank Transfer   cancelled         372
                not cancelled     773
Credit Card     cancelled        2014
                not cancelled    4265
ShopeePay       cancelled         121
                not cancelled     258
Name: orderid, dtype: int64

cancelled orders: 2555
not cancelled orders: 5356
re-ordered :17


Timedelta('15 days 17:23:21.176470')